## Install and import bibraries

In [ ]:
# ! pip install faiss-cpu==1.8.0

   ---------------------------------------- 0.0/14.5 MB ? eta -:--:--
    --------------------------------------- 0.3/14.5 MB ? eta -:--:--
   - -------------------------------------- 0.5/14.5 MB 1.4 MB/s eta 0:00:11
   -- ------------------------------------- 0.8/14.5 MB 1.4 MB/s eta 0:00:10
   ---- ----------------------------------- 1.6/14.5 MB 2.0 MB/s eta 0:00:07
   ----- ---------------------------------- 2.1/14.5 MB 2.1 MB/s eta 0:00:06
   -------- ------------------------------- 3.1/14.5 MB 2.8 MB/s eta 0:00:05
   ------------ --------------------------- 4.5/14.5 MB 3.3 MB/s eta 0:00:04
   -------------- ------------------------- 5.2/14.5 MB 3.5 MB/s eta 0:00:03
   ---------------- ----------------------- 6.0/14.5 MB 3.4 MB/s eta 0:00:03
   ------------------- -------------------- 7.1/14.5 MB 3.6 MB/s eta 0:00:03
   ----------------------- ---------------- 8.4/14.5 MB 3.9 MB/s eta 0:00:02
   --------------------------- ------------ 10.0/14.5 MB 4.2 MB/s eta 0:00:02
   ---------

In [6]:
import numpy as np
import collections
import torch
import faiss
import evaluate

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel
from transformers import AutoModelForQuestionAnswering
from transformers import TrainingArguments
from transformers import Trainer
from tqdm.auto import tqdm

device = torch.device(
    "cuda") if torch.cuda.is_available() else torch.device("cpu")

d:\Asus\Ungdung\Miniconda\workspace\envs\AIOEx\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Download dataset

In [7]:
DATASET_NAME = "rajpurkar/squad_v2"
raw_datasets = load_dataset(DATASET_NAME)
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})

In [8]:
raw_datasets['train'][0]

{'id': '56be85543aeaaa14008c9063',
 'title': 'Beyoncé',
 'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".',
 'question': 'When did Beyonce start becoming popular?',
 'answers': {'text': ['in the late 1990s'], 'answer_start': [269]}}

## Preprocessing

In [9]:
raw_datasets.filter(lambda x: len(x['answers']['text']) > 0)

Filter: 100%|██████████| 11873/11873 [00:00<00:00, 44322.84 examples/s]


DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 86821
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5928
    })
})

## Setup model

In [11]:
MODEL_NAME = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME).to(device)

## Get embedding vector

In [ ]:
# Test tokenizer
text1 = ["This is a very long sentence.",
         "This is a very long sentence."]  # Dài hơn max_length
text2 = "Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981)."

tokens = tokenizer(
    text1,
    max_length=15,
    stride=2,
    truncation=True, 
    padding="max_length"
    )
print(tokens)
print("length inputs_id: ", len(tokens['input_ids']))
print(tokens.sequence_ids())
print(tokenizer.decode(tokens['input_ids'][0]))

{'input_ids': [[101, 2023, 2003, 1037, 2200, 2146, 6251, 1012, 102, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]]}
length inputs_id:  1
[None, 0, 0, 0, 0, 0, 0, 0, None, None, None, None, None, None, None]
[CLS] this is a very long sentence. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


In [37]:
# lấy embeddim của cls tokens để lưu vào vector db
def get_cls_from_hidden(model_output):
    cls_token = model_output.last_hidden_state[:,0]
    return cls_token

In [38]:
def get_embeddings(text_list):
    tokens_list = tokenizer(
        text_list,
        max_length=15,
        stride=2,
        truncation=True,
        padding="max_length",
        return_tensors='pt'
    )
    encoded_token = {k:v.to(device) for k, v in tokens_list.items()}
    model_output = model(**encoded_token)
    
    cls_tokens = get_cls_from_hidden(model_output)
    return cls_tokens

In [39]:
model_output = get_embeddings(text1).size()
model_output

torch.Size([1, 768])

## Build vector database

In [58]:
# Define the name for the new embedding column
EMBEDDING_COLUMN = 'question_embedding'
subset = raw_datasets['train'].select(range(1000))
embeddings_dataset = subset.map(
    lambda x: {
        EMBEDDING_COLUMN: get_embeddings(
            x['question']
        ).detach().cpu().numpy()[0]
    }
)
embeddings_dataset.add_faiss_index(column=EMBEDDING_COLUMN)

100%|██████████| 1/1 [00:00<00:00, 181.56it/s]


Dataset({
    features: ['id', 'title', 'context', 'question', 'answers', 'question_embedding'],
    num_rows: 1000
})

In [59]:
input_question = 'When did Beyonce start becoming popular ?'

input_quest_embedding = get_embeddings([input_question])
input_quest_embedding = input_quest_embedding.cpu().detach().numpy()

TOP_K = 5
scores, samples = embeddings_dataset.get_nearest_examples(
    EMBEDDING_COLUMN, input_quest_embedding, k=TOP_K
)

for idx, score in enumerate(scores):
    print(f'Top {idx + 1}\tScore: {score}')
    print(f'Question: {samples["question"][idx]}')
    print(f'Context: {samples["context"][idx]}')
    print()

Top 1	Score: 0.0
Question: When did Beyonce start becoming popular?
Context: Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".

Top 2	Score: 2.6135406494140625
Question: When did Beyoncé rise to fame?
Context: Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress